# PyTorch를 활용한 멀티 레이어 퍼셉트론 (MLP) 구현

PyTorch를 활용하여,

1. FashionMNIST 데이터셋을 불러온다
2. MLP를 구현한다
3. Classification을 수행한다

### FanshionMNIST 데이터셋 불러오고 살펴보기

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST

In [ ]:
train_dataset = FashionMNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_dataset = FashionMNIST(
    root='./data/',
    train=False,
    download=False,
    transform=transforms.ToTensor()
)

In [ ]:
batch_size = 64

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class_label_map = {
    0: "T-Shirt/Top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(6, 6))
rows, cols = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = np.random.randint(len(train_dataset))
    img, cls = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(class_label_map[cls])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## PyTorch로 MLP 구현하기

#### nn.Linear(input_dim, output_dim)
- Fully connected layer(FC)라고도 부름
- `input_dim`: 입력되는 값의 차원
- `output_dim`: 출력되는 값의 차원
- input_dim=3, output_dim=4라면? 3개의 값이 들어가서 4개의 값으로 나온다!

### 모델 생성하기

![image.png](https://blog.kakaocdn.net/dn/b5XcJp/btqWUpKHPTL/wy9Z1Kg41sn6Nbqe2IKBMk/img.png)

- 3개의 fully connected layer로 구성함 (fc1, fc2, fc3)
- Linear layer인 fc1에 입력하기 위해서 28x28인 샘플을 한 줄로 flatten할 필요가 있음
- fc1, fc2는 ReLU 활성함수를 포함함
- fc3은 최종적으로 10개의 각 클래스에 대한 값을 출력함

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

### 모델 학습하기

In [ ]:
import torch

from torch.optim import SGD

In [ ]:
# 학습을 수행할 디바이스를 설정합니다
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 모델을 디바이스에 올립니다
model = MLP().to(device)

# 옵티마이저를 SGD, 학습률을 0.01로 설정합니다
optimizer = SGD(model.parameters(), lr=0.01)

- 모델을 학습 모드로 둠
- 20회 학습을 반복함
- 데이터로더에서 입력 데이터와 정답 레이블을 차례대로 불러옴
- 불러온 데이터와 레이블을 모델과 같은 디바이스에 올림
- 데이터를 모델에 입력하고 예측 결과를 받음
- 예측 결과를 정답 레이블과 비교하여 로스를 계산함
- 로스로부터 역전파로 그래디언트를 계산함
- 옵티마이저로 업데이트함
- 그래디언트를 0으로 초기화함
- 매 epoch마다의 평균 로스를 출력함

In [ ]:
for i in range(20):
    model.train()
    train_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss
    print(f'[{i+1}] Loss: {train_loss / len(train_loader):.4f}')

### 모델 테스트하기

- 모델을 테스트 모드로 둠
- 그래디언트 계산을 위한 트래킹 기능을 막음
- 데이터로더에서 입력 데이터와 정답 레이블을 차례대로 불러옴
- 불러온 데이터와 레이블을 모델과 같은 디바이스에 올림
- 데이터를 모델에 입력하고 예측 결과를 받음
- 예측 결과를 정답 레이블과 비교하여 로스를 계산함
- 예측 결과를 클래스로 변환함
- 모델의 정확도를 계산함

In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.max(dim=1)[1]
        correct += (pred == target).sum().item()
test_loss /= len(test_dataset)
test_acc = 100 * correct / len(test_dataset)

print(f'Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%')

### 결과 시각화하기

In [ ]:
rows, cols = 6, 6
fig = plt.figure(figsize=(10,10))

model.eval()
for i in range(1, rows * cols + 1):
    data_idx = np.random.randint(len(test_dataset))
    input_img = test_dataset[data_idx][0].unsqueeze(dim=0).to(device)

    output = model(input_img)
    _, argmax = torch.max(output, 1)
    pred = class_label_map[argmax.item()]
    label = class_label_map[test_dataset[data_idx][1]]

    fig.add_subplot(rows, cols, i)
    if pred == label:
        plt.title(pred + ', right')
        cmap = 'Blues'
    else:
        plt.title('N ' + pred + ' B ' +  label)
        cmap = 'Reds'
    plot_img = test_dataset[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')

plt.show()